# Introduction
In the last chapter we had a glimpse of Pandas. In this chapter we will learn about resampling methods and the DataFrame object, which is a powerful tool for financial data analysis.

# Fetching Data
Here we use the Quandl API to retrieve data.



In [38]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np

We will create a Series named "amzn" whose values are Apple's daily closing prices, which are of course indexed by dates:



In [5]:
amzn_table = pdr.get_data_yahoo('AMZN')
amzn = amzn_table['Adj Close']['2020']
print(amzn)

Date
2020-01-02    1898.010010
2020-01-03    1874.969971
2020-01-06    1902.880005
2020-01-07    1906.859985
2020-01-08    1891.969971
                 ...     
2020-12-24    3172.689941
2020-12-28    3283.959961
2020-12-29    3322.000000
2020-12-30    3285.850098
2020-12-31    3256.929932
Name: Adj Close, Length: 253, dtype: float64


Recall that we can fetch a specific data point using series['yyyy-mm-dd']. We can also fetch the data in a specific month using series['yyyy-mm'].

In [7]:
print(amzn['2020-7'])

Date
2020-07-01    2878.699951
2020-07-02    2890.300049
2020-07-06    3057.040039
2020-07-07    3000.120117
2020-07-08    3081.110107
2020-07-09    3182.629883
2020-07-10    3200.000000
2020-07-13    3104.000000
2020-07-14    3084.000000
2020-07-15    3008.870117
2020-07-16    2999.899902
2020-07-17    2961.969971
2020-07-20    3196.840088
2020-07-21    3138.290039
2020-07-22    3099.909912
2020-07-23    2986.550049
2020-07-24    3008.909912
2020-07-27    3055.209961
2020-07-28    3000.330078
2020-07-29    3033.530029
2020-07-30    3051.879883
2020-07-31    3164.679932
Name: Adj Close, dtype: float64


Or in several consecutive months:

In [8]:
amzn['2020-6':'2020-8']

Date
2020-06-01    2471.040039
2020-06-02    2472.409912
2020-06-03    2478.399902
2020-06-04    2460.600098
2020-06-05    2483.000000
                 ...     
2020-08-25    3346.489990
2020-08-26    3441.850098
2020-08-27    3400.000000
2020-08-28    3401.800049
2020-08-31    3450.959961
Name: Adj Close, Length: 65, dtype: float64

.head(N) and .tail(N) are methods for quickly accessing the first or last N elements.

In [10]:
print(amzn.head(), '\n')
print(amzn.tail(10))

Date
2020-01-02    1898.010010
2020-01-03    1874.969971
2020-01-06    1902.880005
2020-01-07    1906.859985
2020-01-08    1891.969971
Name: Adj Close, dtype: float64 

Date
2020-12-17    3236.080078
2020-12-18    3201.649902
2020-12-21    3206.179932
2020-12-22    3206.520020
2020-12-23    3185.270020
2020-12-24    3172.689941
2020-12-28    3283.959961
2020-12-29    3322.000000
2020-12-30    3285.850098
2020-12-31    3256.929932
Name: Adj Close, dtype: float64


# Resampling
**series.resample(freq)** is a class called "DatetimeIndexResampler" which groups data in a Series object into regular time intervals. The argument "freq" determines the length of each interval.

**series.resample.mean()** is a complete statement that groups data into intervals, and then compute the mean of each interval. For example, if we want to aggregate the daily data into monthly data by mean:

In [12]:
by_month = amzn.resample('M').mean()
print (by_month)

Date
2020-01-31    1884.237613
2020-02-29    2066.175267
2020-03-31    1872.310436
2020-04-30    2228.705241
2020-05-31    2394.184021
2020-06-30    2613.545455
2020-07-31    3053.853183
2020-08-31    3249.254755
2020-09-30    3160.748105
2020-10-31    3230.600419
2020-11-30    3141.807996
2020-12-31    3197.750000
Freq: M, Name: Adj Close, dtype: float64


We can also aggregate the data by week:

In [14]:
by_week = amzn.resample('W').mean()
print( by_week.head())

Date
2020-01-05    1886.489990
2020-01-12    1897.184009
2020-01-19    1873.083984
2020-01-26    1881.419983
2020-02-02    1883.797998
Freq: W-SUN, Name: Adj Close, dtype: float64


We can choose almost any frequency by using the format 'nf', where 'n' is an integer and 'f' is M for month, W for week and D for day.

In [15]:
three_day = amzn.resample('3D').mean()
two_week  = amzn.resample('2W').mean()
two_month = amzn.resample('2M').mean()

Besides the mean() method, other methods can also be used with the resampler:



In [16]:
std = amzn.resample('W').std()    # standard deviation
max = amzn.resample('W').max()    # maximum value
min = amzn.resample('W').min()    # minimum value

Often we want to calculate monthly returns of a stock, based on prices on the last day of each month. To fetch those prices, we use the series.resample.agg() method:



In [18]:
last_day = amzn.resample('M').agg(lambda x: x[-1])
print (last_day)

Date
2020-01-31    2008.719971
2020-02-29    1883.750000
2020-03-31    1949.719971
2020-04-30    2474.000000
2020-05-31    2442.370117
2020-06-30    2758.820068
2020-07-31    3164.679932
2020-08-31    3450.959961
2020-09-30    3148.729980
2020-10-31    3036.149902
2020-11-30    3168.040039
2020-12-31    3256.929932
Freq: M, Name: Adj Close, dtype: float64


Or directly calculate the monthly rates of return using the data for the first day and the last day:

In [19]:
monthly_return = amzn.resample('M').agg(lambda x: x[-1]/x[1] - 1)
print (monthly_return)

Date
2020-01-31    0.071334
2020-02-29   -0.080950
2020-03-31    0.021336
2020-04-30    0.289327
2020-05-31    0.054569
2020-06-30    0.115843
2020-07-31    0.094931
2020-08-31    0.099441
2020-09-30   -0.108375
2020-10-31   -0.028432
2020-11-30    0.039243
2020-12-31    0.016669
Freq: M, Name: Adj Close, dtype: float64


In [20]:
print( monthly_return.mean())
print (monthly_return.std())
print (monthly_return.max())

0.04874480696098197
0.10311663928236184
0.28932738004924596


Another two methods frequently used on Series are .diff() and .pct_change(). The former calculates the difference between consecutive elements, and the latter calculates the percentage change.



In [21]:
print (last_day.diff())
print (last_day.pct_change())

Date
2020-01-31           NaN
2020-02-29   -124.969971
2020-03-31     65.969971
2020-04-30    524.280029
2020-05-31    -31.629883
2020-06-30    316.449951
2020-07-31    405.859863
2020-08-31    286.280029
2020-09-30   -302.229980
2020-10-31   -112.580078
2020-11-30    131.890137
2020-12-31     88.889893
Freq: M, Name: Adj Close, dtype: float64
Date
2020-01-31         NaN
2020-02-29   -0.062214
2020-03-31    0.035021
2020-04-30    0.268900
2020-05-31   -0.012785
2020-06-30    0.129567
2020-07-31    0.147114
2020-08-31    0.090461
2020-09-30   -0.087579
2020-10-31   -0.035754
2020-11-30    0.043440
2020-12-31    0.028058
Freq: M, Name: Adj Close, dtype: float64


Notice that we induced a NaN value while calculating percentage changes i.e. returns.

When dealing with NaN values, we usually either removing the data point or fill it with a specific value. Here we fill it with 0:



In [22]:
daily_return = last_day.pct_change()
print( daily_return.fillna(0))

Date
2020-01-31    0.000000
2020-02-29   -0.062214
2020-03-31    0.035021
2020-04-30    0.268900
2020-05-31   -0.012785
2020-06-30    0.129567
2020-07-31    0.147114
2020-08-31    0.090461
2020-09-30   -0.087579
2020-10-31   -0.035754
2020-11-30    0.043440
2020-12-31    0.028058
Freq: M, Name: Adj Close, dtype: float64


Alternatively, we can fill a NaN with the next fitted value. This is called 'backward fill', or 'bfill' in short:

In [23]:
daily_return = last_day.pct_change()
print (daily_return.fillna(method = 'bfill'))

Date
2020-01-31   -0.062214
2020-02-29   -0.062214
2020-03-31    0.035021
2020-04-30    0.268900
2020-05-31   -0.012785
2020-06-30    0.129567
2020-07-31    0.147114
2020-08-31    0.090461
2020-09-30   -0.087579
2020-10-31   -0.035754
2020-11-30    0.043440
2020-12-31    0.028058
Freq: M, Name: Adj Close, dtype: float64


As expected, since there is a 'backward fill' method, there must be a 'forward fill' method, or 'ffill' in short. However we can't use it here because the NaN is the first value.

We can also simply remove NaN values by .dropna()

In [24]:
daily_return = last_day.pct_change().dropna()
print (daily_return)

Date
2020-02-29   -0.062214
2020-03-31    0.035021
2020-04-30    0.268900
2020-05-31   -0.012785
2020-06-30    0.129567
2020-07-31    0.147114
2020-08-31    0.090461
2020-09-30   -0.087579
2020-10-31   -0.035754
2020-11-30    0.043440
2020-12-31    0.028058
Freq: M, Name: Adj Close, dtype: float64


# DataFrame
The DataFrame is the most commonly used data structure in Pandas. It is essentially a table, just like an Excel spreadsheet.

More precisely, a DataFrame is a collection of Series objects, each of which may contain different data types. A DataFrame can be created from various data types: dictionary, 2-D numpy.ndarray, a Series or another DataFrame.

### Create DataFrames
The most common method of creating a DataFrame is passing a dictionary:

In [28]:
dict = {'AMZN': [153.9,  154.29, 152.93, 154.58, 153.17],
        'NVDA': [355.7,  352.12, 356.15, 351.15, 352.39],
        'AMD':  [185.58, 183.97, 182.3, 182.14, 183.79]}
dates = pd.date_range('2020-07-03', periods = 5, freq = 'D')
df = pd.DataFrame(dict, index = dates)
print (df)

              AMZN    NVDA     AMD
2020-07-03  153.90  355.70  185.58
2020-07-04  154.29  352.12  183.97
2020-07-05  152.93  356.15  182.30
2020-07-06  154.58  351.15  182.14
2020-07-07  153.17  352.39  183.79


### Manipulating DataFrames
We can fetch values in a DataFrame by columns and index. Each column in a DataFrame is essentially a Pandas Series. We can fetch a column by square brackets: df['column_name']

If a column name contains no spaces, then we can also use df.column_name to fetch a column:



In [31]:
df = amzn_table
print (df.Close.tail(5), '\n')
print (df['Volume'].tail(5))

Date
2021-03-26    3052.030029
2021-03-29    3075.729980
2021-03-30    3055.290039
2021-03-31    3094.080078
2021-04-01    3136.379883
Name: Close, dtype: float64 

Date
2021-03-26    3306700
2021-03-29    2746000
2021-03-30    2337600
2021-03-31    3091200
2021-04-01    1775072
Name: Volume, dtype: int64


All the methods we applied to a Series index such as iloc[], loc[] and resampling methods, can also be applied to a DataFrame:



In [33]:
amzn_2020 = df['2020']
amzn_month = amzn_2020.resample('M').agg(lambda x: x[-1])
print (amzn_month)

                   High          Low  ...    Volume    Adj Close
Date                                  ...                       
2020-01-31  2055.719971  2002.270020  ...  15567300  2008.719971
2020-02-29  1889.760010  1811.130005  ...   9493800  1883.750000
2020-03-31  1993.020020  1944.010010  ...   5123600  1949.719971
2020-04-30  2475.000000  2396.010010  ...   9534600  2474.000000
2020-05-31  2442.370117  2398.199951  ...   3529300  2442.370117
2020-06-30  2769.629883  2675.030029  ...   3769700  2758.820068
2020-07-31  3246.820068  3151.000000  ...   8085500  3164.679932
2020-08-31  3495.000000  3405.000000  ...   4185900  3450.959961
2020-09-30  3212.879883  3133.989990  ...   4896100  3148.729980
2020-10-31  3167.000000  3019.000000  ...   8386400  3036.149902
2020-11-30  3228.389893  3125.550049  ...   4063900  3168.040039
2020-12-31  3282.919922  3241.199951  ...   2957200  3256.929932

[12 rows x 6 columns]


We may select certain columns of a DataFrame using their names:

In [35]:
amzn_bar = amzn_month[['Open', 'High', 'Low', 'Close']]
print (amzn_bar)

                   Open         High          Low        Close
Date                                                          
2020-01-31  2051.469971  2055.719971  2002.270020  2008.719971
2020-02-29  1814.630005  1889.760010  1811.130005  1883.750000
2020-03-31  1964.349976  1993.020020  1944.010010  1949.719971
2020-04-30  2419.840088  2475.000000  2396.010010  2474.000000
2020-05-31  2415.939941  2442.370117  2398.199951  2442.370117
2020-06-30  2685.070068  2769.629883  2675.030029  2758.820068
2020-07-31  3244.000000  3246.820068  3151.000000  3164.679932
2020-08-31  3408.989990  3495.000000  3405.000000  3450.959961
2020-09-30  3141.139893  3212.879883  3133.989990  3148.729980
2020-10-31  3157.750000  3167.000000  3019.000000  3036.149902
2020-11-30  3208.479980  3228.389893  3125.550049  3168.040039
2020-12-31  3275.000000  3282.919922  3241.199951  3256.929932


We can even specify both rows and columns using loc[]. The row indices and column names are separated by a comma:

In [36]:
print( amzn_month.loc['2020-03':'2020-06', ['Open', 'High', 'Low', 'Close']])

                   Open         High          Low        Close
Date                                                          
2020-03-31  1964.349976  1993.020020  1944.010010  1949.719971
2020-04-30  2419.840088  2475.000000  2396.010010  2474.000000
2020-05-31  2415.939941  2442.370117  2398.199951  2442.370117
2020-06-30  2685.070068  2769.629883  2675.030029  2758.820068


The subset methods in DataFrame is quite useful. By writing logical statements in square brackets, we can make customized subsets:



In [39]:
above = amzn_bar[amzn_bar.Close > np.mean(amzn_bar.Close)]
print(above)


                   Open         High          Low        Close
Date                                                          
2020-06-30  2685.070068  2769.629883  2675.030029  2758.820068
2020-07-31  3244.000000  3246.820068  3151.000000  3164.679932
2020-08-31  3408.989990  3495.000000  3405.000000  3450.959961
2020-09-30  3141.139893  3212.879883  3133.989990  3148.729980
2020-10-31  3157.750000  3167.000000  3019.000000  3036.149902
2020-11-30  3208.479980  3228.389893  3125.550049  3168.040039
2020-12-31  3275.000000  3282.919922  3241.199951  3256.929932


### Data Validation
As mentioned, all methods that apply to a Series can also be applied to a DataFrame. Here we add a new column to an existing DataFrame:

In [40]:
amzn_bar['rate_return'] = amzn_bar.Close.pct_change()
print (amzn_bar)

                   Open         High          Low        Close  rate_return
Date                                                                       
2020-01-31  2051.469971  2055.719971  2002.270020  2008.719971          NaN
2020-02-29  1814.630005  1889.760010  1811.130005  1883.750000    -0.062214
2020-03-31  1964.349976  1993.020020  1944.010010  1949.719971     0.035021
2020-04-30  2419.840088  2475.000000  2396.010010  2474.000000     0.268900
2020-05-31  2415.939941  2442.370117  2398.199951  2442.370117    -0.012785
2020-06-30  2685.070068  2769.629883  2675.030029  2758.820068     0.129567
2020-07-31  3244.000000  3246.820068  3151.000000  3164.679932     0.147114
2020-08-31  3408.989990  3495.000000  3405.000000  3450.959961     0.090461
2020-09-30  3141.139893  3212.879883  3133.989990  3148.729980    -0.087579
2020-10-31  3157.750000  3167.000000  3019.000000  3036.149902    -0.035754
2020-11-30  3208.479980  3228.389893  3125.550049  3168.040039     0.043440
2020-12-31  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Here the calculation introduced a NaN value. If the DataFrame is large, we would not be able to observe it. isnull() provides a convenient way to check abnormal values.

In [41]:
missing = amzn_bar.isnull()
print (missing)
print ('---------------------------------------------')
print (missing.describe())

             Open   High    Low  Close  rate_return
Date                                               
2020-01-31  False  False  False  False         True
2020-02-29  False  False  False  False        False
2020-03-31  False  False  False  False        False
2020-04-30  False  False  False  False        False
2020-05-31  False  False  False  False        False
2020-06-30  False  False  False  False        False
2020-07-31  False  False  False  False        False
2020-08-31  False  False  False  False        False
2020-09-30  False  False  False  False        False
2020-10-31  False  False  False  False        False
2020-11-30  False  False  False  False        False
2020-12-31  False  False  False  False        False
---------------------------------------------
         Open   High    Low  Close rate_return
count      12     12     12     12          12
unique      1      1      1      1           2
top     False  False  False  False       False
freq       12     12     12     12    

The row labelled "unique" indicates the number of unique values in each column. Since the "rate_return" column has 2 unique values, it has at least one missing value.

We can deduce the number of missing values by comparing "count" with "freq". There are 12 counts and 11 False values, so there is one True value which corresponds to the missing value.

We can also find the rows with missing values easily:

In [42]:
print (missing[missing.rate_return == True])

             Open   High    Low  Close  rate_return
Date                                               
2020-01-31  False  False  False  False         True


Usually when dealing with missing data, we either delete the whole row or fill it with some value. As we introduced in the Series chapter, the same method dropna() and fillna() can be applied to a DataFrame.



In [43]:
drop = amzn_bar.dropna()
print (drop)
print ('\n--------------------------------------------------\n')
fill = amzn_bar.fillna(0)
print (fill)


                   Open         High          Low        Close  rate_return
Date                                                                       
2020-02-29  1814.630005  1889.760010  1811.130005  1883.750000    -0.062214
2020-03-31  1964.349976  1993.020020  1944.010010  1949.719971     0.035021
2020-04-30  2419.840088  2475.000000  2396.010010  2474.000000     0.268900
2020-05-31  2415.939941  2442.370117  2398.199951  2442.370117    -0.012785
2020-06-30  2685.070068  2769.629883  2675.030029  2758.820068     0.129567
2020-07-31  3244.000000  3246.820068  3151.000000  3164.679932     0.147114
2020-08-31  3408.989990  3495.000000  3405.000000  3450.959961     0.090461
2020-09-30  3141.139893  3212.879883  3133.989990  3148.729980    -0.087579
2020-10-31  3157.750000  3167.000000  3019.000000  3036.149902    -0.035754
2020-11-30  3208.479980  3228.389893  3125.550049  3168.040039     0.043440
2020-12-31  3275.000000  3282.919922  3241.199951  3256.929932     0.028058

-----------

### DataFrame Concat
We have seen how to extract a Series from a dataFrame. Now we need to consider how to merge a Series or a DataFrame into another one.

In Pandas, the function concat() allows us to merge multiple Series into a DataFrame:

In [44]:
s1 = pd.Series([153.9,  154.29, 152.93, 154.58, 153.17], name = 'AMZN')
s2 = pd.Series([355.7,  352.12, 356.15, 351.15, 352.39], name = 'NVDA')
data_frame = pd.concat([s1, s2], axis = 1)
print (data_frame)

     AMZN    NVDA
0  153.90  355.70
1  154.29  352.12
2  152.93  356.15
3  154.58  351.15
4  153.17  352.39


The "axis = 1" parameter will join two DataFrames by columns:

In [46]:
log_price = np.log(amzn_bar.Close)
log_price.name = 'log_price'
print (log_price)
print ('\n--------------------------------------------\n')
concat = pd.concat([amzn_bar, log_price], axis = 1)
print (concat)

Date
2020-01-31    7.605253
2020-02-29    7.541020
2020-03-31    7.575441
2020-04-30    7.813592
2020-05-31    7.800724
2020-06-30    7.922558
2020-07-31    8.059807
2020-08-31    8.146408
2020-09-30    8.054754
2020-10-31    8.018346
2020-11-30    8.060868
2020-12-31    8.088540
Freq: M, Name: log_price, dtype: float64

--------------------------------------------

                   Open         High  ...  rate_return  log_price
Date                                  ...                        
2020-01-31  2051.469971  2055.719971  ...          NaN   7.605253
2020-02-29  1814.630005  1889.760010  ...    -0.062214   7.541020
2020-03-31  1964.349976  1993.020020  ...     0.035021   7.575441
2020-04-30  2419.840088  2475.000000  ...     0.268900   7.813592
2020-05-31  2415.939941  2442.370117  ...    -0.012785   7.800724
2020-06-30  2685.070068  2769.629883  ...     0.129567   7.922558
2020-07-31  3244.000000  3246.820068  ...     0.147114   8.059807
2020-08-31  3408.989990  3495.000000 

We can also join two DataFrames by rows. Consider these two DataFrames:

In [50]:
df_volume = amzn_table.loc['2020-10':'2020-04', ['Volume']].resample('M').agg(lambda x: x[-1])
print (df_volume)
print ('\n-------------------------------------------\n')
df_2020 = amzn_table.loc['2020-10':'2020-04', ['Open', 'High', 'Low', 'Close']].resample('M').agg(lambda x: x[-1])
print (df_2020)

Empty DataFrame
Columns: [Volume]
Index: []

-------------------------------------------

Empty DataFrame
Columns: [Open, High, Low, Close]
Index: []


Now we merge the DataFrames with our DataFrame 'aapl_bar'

In [51]:
concat = pd.concat([amzn_bar, df_volume], axis = 1)
print(concat)

                   Open         High  ...  rate_return  Volume
Date                                  ...                     
2020-01-31  2051.469971  2055.719971  ...          NaN     NaN
2020-02-29  1814.630005  1889.760010  ...    -0.062214     NaN
2020-03-31  1964.349976  1993.020020  ...     0.035021     NaN
2020-04-30  2419.840088  2475.000000  ...     0.268900     NaN
2020-05-31  2415.939941  2442.370117  ...    -0.012785     NaN
2020-06-30  2685.070068  2769.629883  ...     0.129567     NaN
2020-07-31  3244.000000  3246.820068  ...     0.147114     NaN
2020-08-31  3408.989990  3495.000000  ...     0.090461     NaN
2020-09-30  3141.139893  3212.879883  ...    -0.087579     NaN
2020-10-31  3157.750000  3167.000000  ...    -0.035754     NaN
2020-11-30  3208.479980  3228.389893  ...     0.043440     NaN
2020-12-31  3275.000000  3282.919922  ...     0.028058     NaN

[12 rows x 6 columns]


By default the DataFrame are joined with all of the data. This default options results in zero information loss. We can also merge them by intersection, this is called 'inner join':



In [52]:
concat = pd.concat([amzn_bar, df_volume], axis = 1, join = 'inner')
print (concat)

Empty DataFrame
Columns: [Open, High, Low, Close, rate_return, Volume]
Index: []


Only the intersection part was left if use 'inner join' method. Now let's try to append a DataFrame to another one:



In [53]:
append = amzn_bar.append(df_2020)
print (append)

                   Open         High          Low        Close  rate_return
Date                                                                       
2020-01-31  2051.469971  2055.719971  2002.270020  2008.719971          NaN
2020-02-29  1814.630005  1889.760010  1811.130005  1883.750000    -0.062214
2020-03-31  1964.349976  1993.020020  1944.010010  1949.719971     0.035021
2020-04-30  2419.840088  2475.000000  2396.010010  2474.000000     0.268900
2020-05-31  2415.939941  2442.370117  2398.199951  2442.370117    -0.012785
2020-06-30  2685.070068  2769.629883  2675.030029  2758.820068     0.129567
2020-07-31  3244.000000  3246.820068  3151.000000  3164.679932     0.147114
2020-08-31  3408.989990  3495.000000  3405.000000  3450.959961     0.090461
2020-09-30  3141.139893  3212.879883  3133.989990  3148.729980    -0.087579
2020-10-31  3157.750000  3167.000000  3019.000000  3036.149902    -0.035754
2020-11-30  3208.479980  3228.389893  3125.550049  3168.040039     0.043440
2020-12-31  

'Append' is essentially to concat two DataFrames by axis = 0, thus here is an alternative way to append:



In [55]:
concat = pd.concat([amzn_bar, df_2020], axis = 0)
print (concat)

                   Open         High          Low        Close  rate_return
Date                                                                       
2020-01-31  2051.469971  2055.719971  2002.270020  2008.719971          NaN
2020-02-29  1814.630005  1889.760010  1811.130005  1883.750000    -0.062214
2020-03-31  1964.349976  1993.020020  1944.010010  1949.719971     0.035021
2020-04-30  2419.840088  2475.000000  2396.010010  2474.000000     0.268900
2020-05-31  2415.939941  2442.370117  2398.199951  2442.370117    -0.012785
2020-06-30  2685.070068  2769.629883  2675.030029  2758.820068     0.129567
2020-07-31  3244.000000  3246.820068  3151.000000  3164.679932     0.147114
2020-08-31  3408.989990  3495.000000  3405.000000  3450.959961     0.090461
2020-09-30  3141.139893  3212.879883  3133.989990  3148.729980    -0.087579
2020-10-31  3157.750000  3167.000000  3019.000000  3036.149902    -0.035754
2020-11-30  3208.479980  3228.389893  3125.550049  3168.040039     0.043440
2020-12-31  

Please note that if the two DataFrame have some columns with the same column names, these columns are considered to be the same and will be merged. It's very important to have the right column names. If we change a column names here:

In [56]:
df_2020.columns = ['Change', 'High', 'Low', 'Close']
concat = pd.concat([amzn_bar, df_2020], axis = 0)
print (concat)


                   Open         High  ...  rate_return  Change
Date                                  ...                     
2020-01-31  2051.469971  2055.719971  ...          NaN     NaN
2020-02-29  1814.630005  1889.760010  ...    -0.062214     NaN
2020-03-31  1964.349976  1993.020020  ...     0.035021     NaN
2020-04-30  2419.840088  2475.000000  ...     0.268900     NaN
2020-05-31  2415.939941  2442.370117  ...    -0.012785     NaN
2020-06-30  2685.070068  2769.629883  ...     0.129567     NaN
2020-07-31  3244.000000  3246.820068  ...     0.147114     NaN
2020-08-31  3408.989990  3495.000000  ...     0.090461     NaN
2020-09-30  3141.139893  3212.879883  ...    -0.087579     NaN
2020-10-31  3157.750000  3167.000000  ...    -0.035754     NaN
2020-11-30  3208.479980  3228.389893  ...     0.043440     NaN
2020-12-31  3275.000000  3282.919922  ...     0.028058     NaN

[12 rows x 6 columns]


Since the column name of 'Open' has been changed, the new DataFrame has an new column named 'Change'.

# Summary
Hereby we introduced the most import part of python: resampling and DataFrame manipulation. We only introduced the most commonly used method in Financial data analysis. There are also many methods used in data mining, which are also beneficial. You can always check the Pandas official documentations for help.